In [1]:
import numpy as np
from qiskit.opflow import X, Z, Y, I

In [2]:
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP
from qiskit import BasicAer, Aer
from qiskit.algorithms import VQE

Start of 29/11/22:

1. define hamiltonian for 1D and then 2D J1-J2 model ->
    a. exact ground state
    b. state vector simulation runs
2. define measurements using actual quantum circuit
3. construct ansatz
4. VQE and optimization

psi
<psi|H|psi>

In [3]:
def getHamiltonian_J1J2_2D(m, n, J1, J2, h=0):
    """
        m x n lattice of spins
        J1: nearest neighbour interaction
        J2: next to nearest neighbour interaction
        h: magnetic field, taken to be 0 for now
        
        H = - J1 ΣSi.Sj - J2 ΣSi.Sj - h ΣSi
    """
    N = m*n
    # H = np.zeros((2**N, 2**N))
    H1 = 0
    H2 = 0
    
    def getIndex(i, j):
        """
            mapping for spins: i,j (0<=i<m, 0<=j<n)
                maps to (i*n+j)
        """
        assert 0<=i<m and 0<=j<n
        return i*n+j
    
    # pauli_x = np.array([[0, 1], [1, 0]])
    # pauli_y = np.array([[0, -1j], [1j, 0]])
    # pauli_z = np.array([[1, 0], [0, -1]])
    
    # contribution of nearest neighbour, spins X, Y and Z:
    # for pauli in [pauli_x, pauli_y, pauli_z]:
    for pauli in [X, Y, Z]:
        for i in range(m):
            for j in range(n):
                indexCurr = getIndex(i,j)
                index_East = getIndex(i,(j+1)%n)
                total = 0
                for ind in range(0, N):
                    if (ind == indexCurr 
                        or ind == index_East):
                        curr = pauli
                    else:
                        # curr = np.identity((2,2))
                        curr = I
                    if not total:
                        total = curr
                    else:
                        # total = np.kron(total, curr)
                        total = total^curr
                # print(total)
                if not H1:
                    H1 = total
                else:
                    H1 += total
                index_South = getIndex((i+1)%m,j)
                total = 0
                for ind in range(0, N):
                    if (ind == indexCurr or ind == index_South):
                        curr = pauli
                    else:
                        curr = I
                    if not total:
                        total = curr
                    else:
                        # total = np.kron(total, curr)
                        total = total^curr
                H1 += total
                
         
    # contribution of next to nearest neighbour, spins X, Y and Z:
    # for pauli in [pauli_x, pauli_y, pauli_z]:
    for pauli in [X, Y, Z]:
        for i in range(m):
            for j in range(n):
                indexCurr = getIndex(i,j)
                index_NorthEast = getIndex((i-1)%m,(j+1)%n)
                total = 0
                for ind in range(0, N):
                    if (ind == indexCurr or ind == index_NorthEast):
                        curr = pauli
                    else:
                        # curr = np.identity((2,2))
                        curr = I
                    if not total:
                        total = curr
                    else:
                        # total = np.kron(total, curr)
                        total = total^curr
                # print(total)
                if not H2:
                    H2 = total
                else:
                    H2 += total
                index_SouthEast = getIndex((i+1)%m,(j+1)%n)
                total = 0
                for ind in range(0, N):
                    if (ind == indexCurr or ind == index_SouthEast):
                        curr = pauli
                    else:
                        curr = I
                    if not total:
                        total = curr
                    else:
                        # total = np.kron(total, curr)
                        total = total^curr
                H2 += total
                
    return H1*J1 + J2*H2
            
            
# corner cases: 1D, 2x2 don't work
# 1D takes neighbours that don't exist
# and 2D has multiple repetitions of same bonds

In [4]:
getHamiltonian_J1J2_2D(3,3,1,0.5)

PauliSumOp(SparsePauliOp(['XXIIIIIII', 'XIIXIIIII', 'IXXIIIIII', 'IXIIXIIII', 'XIXIIIIII', 'IIXIIXIII', 'IIIXXIIII', 'IIIXIIXII', 'IIIIXXIII', 'IIIIXIIXI', 'IIIXIXIII', 'IIIIIXIIX', 'IIIIIIXXI', 'XIIIIIXII', 'IIIIIIIXX', 'IXIIIIIXI', 'IIIIIIXIX', 'IIXIIIIIX', 'YYIIIIIII', 'YIIYIIIII', 'IYYIIIIII', 'IYIIYIIII', 'YIYIIIIII', 'IIYIIYIII', 'IIIYYIIII', 'IIIYIIYII', 'IIIIYYIII', 'IIIIYIIYI', 'IIIYIYIII', 'IIIIIYIIY', 'IIIIIIYYI', 'YIIIIIYII', 'IIIIIIIYY', 'IYIIIIIYI', 'IIIIIIYIY', 'IIYIIIIIY', 'ZZIIIIIII', 'ZIIZIIIII', 'IZZIIIIII', 'IZIIZIIII', 'ZIZIIIIII', 'IIZIIZIII', 'IIIZZIIII', 'IIIZIIZII', 'IIIIZZIII', 'IIIIZIIZI', 'IIIZIZIII', 'IIIIIZIIZ', 'IIIIIIZZI', 'ZIIIIIZII', 'IIIIIIIZZ', 'IZIIIIIZI', 'IIIIIIZIZ', 'IIZIIIIIZ', 'XIIIIIIXI', 'XIIIXIIII', 'IXIIIIIIX', 'IXIIIXIII', 'IIXIIIXII', 'IIXXIIIII', 'IXIXIIIII', 'IIIXIIIXI', 'IIXIXIIII', 'IIIIXIIIX', 'XIIIIXIII', 'IIIIIXXII', 'IIIIXIXII', 'IXIIIIXII', 'IIIIIXIXI', 'IIXIIIIXI', 'IIIXIIIIX', 'XIIIIIIIX', 'YIIIIIIYI', 'YIIIYIIII', 'IYIIIIIIY',

In [5]:
ham = getHamiltonian_J1J2_2D(3,3,1,0.5)
M = ham.to_matrix()

# TODO: approximate the eigenvalue for larger systems
exactEnergy = np.min(np.linalg.eigvals(M))
print(exactEnergy)

(-13.938003499857201+5.367606511339193e-16j)


Plan for this week and next:

Friday (02/12/22):
- Basic parametrized ansatz for 3x3 lattice, run the simulator, use statevector simulation and matrix form of Hamiltonian to obtain a result

0/1
psi 

**Make sure changing parameters actually changes the output**

Tuesday (06/12/22):
- Try VQE on the parametrized ansatz